# Python for Data Scientist
## From data manipulation to machine learning

# scikit-learn

* classification
* regression
* clustering
* preprocessing
* ...

In [ ]:
import seaborn as sns

from sklearn import svm
from sklearn.model_selection import train_test_split

### Carico il dataset dei passeggeri del titanic da seaborn

In [ ]:
original_data = sns.load_dataset("titanic")
original_data

### Creo una copia del dataset in modo da mantenere i dati orginali

In [ ]:
data = original_data.copy()

In [ ]:
data[columns].describe()

### Converto alcune colonne di tipo categorico in tipo numerico

In [ ]:
# Nota: se eseguita due volte, questa operazione rende tutta la colonna NaN
data.sex = data.sex.map({"male": -1, "female": 1})
data.sex.unique()

In [ ]:
# nota: class è già presente in formato numerico come classe pclass
data["class"] = data["class"].map({"First": 1, "Second": 2, "Third": 3})
data["class"].unique()

### Elimino alcuni valori NaN dalla colonna `age` impostandoli al valor medio
Questo non è ne l'unico ne l'ottimale approccio per "eliminare" i valori NaN. Una strategia di pulitura dei dati mancanti va sempre stabilita in base al tipo di dataset e ai propri obiettivi e con una buona dose di buon senso. 

**Nota**: la funzione `count()` ritorna il numero di valori diversi da **NaN**/**None**

In [ ]:
data.age.count?
# len(data.age[data.age.isna()])
data.age.count()

In [ ]:
mean_age = data.age[data.age.notna()].mean()
data.age.fillna(mean_age, inplace=True)
data.age.count()

### Seleziono e normalizzo le colonne che daremo in pasto alla SVM 
Buona norma quando si usano algoritmi come le SVM è quello di normalizzare i dati in modo che la scala dei numeri passati non vada a intaccare l'algoritmo

Saltare questo passaggio causa un peggioramento visibile delle performance nel caso di una SVM

In [ ]:
columns = ["pclass", "age", "sibsp", "parch", "fare", "sex"]

for column in columns:
    col = data[column]
    data[column] = (col - col.mean()) / col.std()

In [ ]:
data[columns].describe()

### Separo i dati preparati in due insiemi
#### un insieme di train (usato per addestrare il modello)
#### un insieme di test (usato per validare le performance del mio modello)

In [ ]:
train, test = train_test_split(data, test_size=0.2)

len(train), len(test)

### Creo il mio modello SVM di tipo classificatore (SVC) e lo addestro sull'insieme di train

In [ ]:
classifier = svm.SVC()

classifier.fit(train[columns], train.survived)

### Provo a predire la sopravvivenza dei passeggeri nell'insieme di test
Per convenienza la aggiungo come colonna del dataset

In [ ]:
test["predict"] = classifier.predict(test[columns])

In [ ]:
test

In [ ]:
print(f"Predict correctly: {len(test[test.predict == test.survived])}/{len(test)}")
test[test.predict != test.survived]